In [1]:
import pandas as pd
import utils
import datetime as dt
from dateutil.parser import *

In [2]:
df_trades = pd.read_pickle("USD_JPY_H4_trades.pkl")

In [3]:
df_trades.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,...,mid_l_prev,ask_h_prev,bid_l_prev,RANGE_prev,DIRECTION,DIRECTION_prev,SIGNAL,ENTRY,STOPLOSS,TAKEPROFIT
4,2022-01-03T18:00:00.000000000Z,4419,115.318,115.358,115.263,115.350,115.312,115.350,115.256,115.343,...,115.138,115.377,115.132,0.232,1,1,1,115.4002,115.3074,115.5858
9,2022-01-04T14:00:00.000000000Z,17583,116.260,116.338,115.955,116.173,116.253,116.331,115.948,116.166,...,115.811,116.359,115.804,0.541,-1,1,1,116.4131,116.1967,116.8459
10,2022-01-04T18:00:00.000000000Z,5432,116.172,116.184,116.080,116.148,116.164,116.177,116.074,116.137,...,115.955,116.345,115.948,0.383,-1,-1,-1,115.9097,116.0629,115.6033
20,2022-01-06T10:00:00.000000000Z,14994,115.750,115.892,115.697,115.750,115.743,115.885,115.690,115.745,...,115.658,115.963,115.652,0.298,-1,-1,-1,115.6222,115.7414,115.3838
24,2022-01-07T02:00:00.000000000Z,6340,115.898,115.939,115.833,115.914,115.891,115.931,115.827,115.908,...,115.830,116.056,115.785,0.218,1,1,1,116.0778,115.9906,116.2522


In [4]:
pair = "USD_JPY"

In [5]:
df_raw = pd.read_pickle(utils.get_his_data_filename(pair, "M5"))

In [6]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df_raw.columns if x not in non_cols]
df_raw[mod_cols] = df_raw[mod_cols].apply(pd.to_numeric)

In [7]:
df_trades["time"] = [parse(x) for x in df_trades.time] 
df_raw["time"] = [parse(x) for x in df_raw.time] 

In [8]:
df_trades["next"] = df_trades["time"].shift(-1)

In [9]:
df_trades['trade_end'] = df_trades.next + dt.timedelta(hours=3, minutes=55)
df_trades['trade_start'] = df_trades.time + dt.timedelta(hours=4)

In [10]:
df_trades[['time', 'next', 'trade_end', 'trade_start']].head()

,time,next,trade_end,trade_start
4,2022-01-03 18:00:00+00:00,2022-01-04 14:00:00+00:00,2022-01-04 17:55:00+00:00,2022-01-03 22:00:00+00:00
9,2022-01-04 14:00:00+00:00,2022-01-04 18:00:00+00:00,2022-01-04 21:55:00+00:00,2022-01-04 18:00:00+00:00
10,2022-01-04 18:00:00+00:00,2022-01-06 10:00:00+00:00,2022-01-06 13:55:00+00:00,2022-01-04 22:00:00+00:00
20,2022-01-06 10:00:00+00:00,2022-01-07 02:00:00+00:00,2022-01-07 05:55:00+00:00,2022-01-06 14:00:00+00:00
24,2022-01-07 02:00:00+00:00,2022-01-10 18:00:00+00:00,2022-01-10 21:55:00+00:00,2022-01-07 06:00:00+00:00


In [11]:
df_trades.dropna(inplace=True)
df_trades.reset_index(drop=True, inplace=True)

In [12]:
def signal_text(signal):
    if signal == 1:
        return 'BUY'
    elif signal == -1:
        return 'SELL'
    return 'NONE'

def triggered(direction, current_price, signal_price):
    if direction == 1 and current_price > signal_price:
        return True
    elif direction == -1 and current_price < signal_price:
        return True
    return False

def end_hit_calc(direction, SL, price, start_price):
    delta = price - start_price
    full_delta = start_price - SL
    fraction = abs(delta / full_delta)
   
    if direction == 1 and price >= start_price:
        return fraction
    elif direction == 1 and price < start_price:
        return -fraction
    elif direction == -1 and price <= start_price:
        return fraction
    elif direction == -1 and price > start_price:
        return -fraction


def process_buy(TP, SL, ask_prices, bid_prices, entry_price):
    for index, price in enumerate(ask_prices):
        if triggered(1, price, entry_price) == True:
            for live_price in bid_prices[index:]:
                if live_price >= TP:
                    return 2.0
                elif live_price <= SL:
                    return -1.0
            return end_hit_calc(1, SL, live_price, entry_price)
    return 0.0

def process_sell(TP, SL, ask_prices, bid_prices, entry_price):
    for index, price in enumerate(bid_prices):
        if triggered(-1, price, entry_price) == True:
            for live_price in ask_prices[index:]:
                if live_price <= TP:
                    return 2.0
                elif live_price >= SL:
                    return -1.0
            return end_hit_calc(-1, SL, live_price, entry_price)
    return 0.0


In [13]:
total = 0
for index, row in df_trades.iterrows():
    m5_data = df_raw[(df_raw.time >= row.trade_start) & (df_raw.time <= row.trade_end)]
    if row.SIGNAL == 1:
        r = process_buy(row.TAKEPROFIT, row.STOPLOSS, m5_data.ask_c.values, m5_data.bid_c.values, row.ENTRY)
        total += r
    else:
        r = process_sell(row.TAKEPROFIT, row.STOPLOSS, m5_data.ask_c.values, m5_data.bid_c.values, row.ENTRY)
        total += r
print(total)

76.1107767427418
